In [1]:
import requests
import json


In [2]:
API_URL = "http://127.0.0.1/api/v1"

In [3]:
def get_and_show(url):
    
    r = requests.get(url)
    if r.status_code == requests.codes.ok:
        print(json.dumps(r.json(), indent = 2))
    else:
        print(f"{r.status_code}\n{json.dumps(r.json(), indent = 2)}")

In [4]:
get_and_show(f"{API_URL}/health")


{
  "status": "OK",
  "deployment_type": "local"
}


In [5]:
# Show deployments

get_and_show(f"{API_URL}/ground-truth/deployments/")

{
  "total": 5,
  "items": [
    {
      "id": "da6e1977-5081-4d67-ade6-69713a2c4bc8",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-11T15:45:50.641708Z",
      "updated_at": null
    },
    {
      "id": "83dde201-1a5e-41c4-bc3d-71ab48ccda36",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-11T15:45:56.098211Z",
      "updated_at": null
    },
    {
      "id": "cbb63174-b5f0-46f6-9ec1-aa7578ecf544",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-12T14:17:57.325988Z",
      "updated_at": null
    },
    {
      "id": "6955c1d4-2819-4178-8da8-821650f9f27e",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-12T14:18:50.715925Z",
      "updated_at": nul

In [9]:
payload =  """I recently added a beta release, version 7.0b1, of my extension, Clippings for Thunderbird, and selected the option to self-host it so that I can make it available to testers separately while regular users continue to see version 6.3.5, the current release version. However, the beta release is now incorrectly appearing in the Add-ons for Thunderbird public listing. URL to ATN listing: https://addons.thunderbird.net/en-US/thunderbird/addon/clippings-tb/. Thanks for the report, it is best to file an issue with the addons-server: https://github.com/thunderbird/addons-server I have not looked at it in detail, but self-hosted add-ons need an update_url entry in their manifest and that should prevent it from being accepted on ATN. Since we do not sign add-ons, self-hosted add-ons do not need to be submitted to ATN at all.My best-practice advice is:
- remove the beta version from ATN,
- create a dedicated branch/repo which holds the update information (either a branch in each add-on repo you want to self-host, or a single update-repo with the information for all your add-ons)
- host the XPI files either as an "beta" asset in a "github release", or directly as a file in the repo (I think the asset is the better choice),
- make sure that the manifest of your self-hosted XPI points to the correct update.json. John I've removed the beta version of my extension from ATN, and the listing for my extension now shows the current stable release. Thanks for your help!"""

In [14]:
requests.post(f"{API_URL}/ground-truth/deployments/19030812-2b05-4c39-a861-8d9e04aa0d47", json={"text": payload}).json()

{'text': {'result': 'Clippings for Thunderbird self-hosts a beta release of version 7.0b1. The beta release is now incorrectly appearing in the Add-ons for Thunderbirds public listing. John has removed the beta version from ATN, and the listing for my extension now shows the current stable release.'}}